In [60]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline

In [21]:
df = pd.read_csv('data/jiebao_all_user_processed.csv')
df.head()

,Unnamed: 0,user_id,follow_count,fans_count,gender,location,level,regis_year,regis_month,regis_day,properties,mileage,car_like,excellent_post_count,all_post_count,label
0,0,oden123,0,0,2,1,1,2018,3,3,1,4430.0,1,0,0,True
1,1,生活1934626,2,4,2,2,1,2012,5,5,1,4270.0,2,0,0,True
2,2,wangzi1125,0,1,2,3,1,2017,6,6,1,2090.0,3,0,1,True
3,3,房产专家谢广财,3,2,2,4,1,2014,2,2,1,1120.0,4,0,0,True
4,4,南宫晗笑,4,5,2,5,1,2017,7,7,1,740.0,1,0,14,True


In [22]:
df.dtypes

Unnamed: 0                int64
user_id                  object
follow_count              int64
fans_count                int64
gender                    int64
location                  int64
level                     int64
regis_year                int64
regis_month               int64
regis_day                 int64
properties                int64
mileage                 float64
car_like                  int64
excellent_post_count      int64
all_post_count            int64
label                      bool
dtype: object

In [23]:
x = df.drop(columns=['Unnamed: 0','user_id', 'label'])
x.head()

,follow_count,fans_count,gender,location,level,regis_year,regis_month,regis_day,properties,mileage,car_like,excellent_post_count,all_post_count
0,0,0,2,1,1,2018,3,3,1,4430.0,1,0,0
1,2,4,2,2,1,2012,5,5,1,4270.0,2,0,0
2,0,1,2,3,1,2017,6,6,1,2090.0,3,0,1
3,3,2,2,4,1,2014,2,2,1,1120.0,4,0,0
4,4,5,2,5,1,2017,7,7,1,740.0,1,0,14


In [34]:
y = df['label'].astype('uint8')
y.describe()

count    2961.000000
mean        0.083755
std         0.277068
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: label, dtype: float64

In [52]:
x, y = np.array(x), np.array(y)
x.shape, y.shape

((2961, 13), (2961,))

In [53]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,  random_state = 42)

In [54]:
#y_test.describe()

In [55]:
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5,  random_state = 42)

In [64]:
clf = SVC(C=0.8, kernel='rbf')

In [65]:
x_train.shape, y_train.shape

((2072, 13), (2072,))

In [66]:
clf.fit(x_train, y_train)

SVC(C=0.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [67]:
pred = clf.predict(x_val)

In [68]:
print(classification_report(y_val, pred))

             precision    recall  f1-score   support

          0       0.91      1.00      0.96       406
          1       0.00      0.00      0.00        38

avg / total       0.84      0.91      0.87       444



/Users/pierre_pc/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
